In [1]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
from langchain_groq import ChatGroq
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [2]:
load_dotenv()

True

In [3]:
key = os.getenv("mykey")

In [5]:
llm = ChatGroq(api_key=key,model = "gemma-7b-it",temperature=0.3)

In [6]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [7]:
"""
number=5 
subject="data science"
tone="simple"
"""

'\nnumber=5 \nsubject="data science"\ntone="simple"\n'

In [8]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [9]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [10]:
quiz_generation_prompt

PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. \nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make {number} MCQs\n### RESPONSE_JSON\n{response_json}\n\n')

In [11]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

d:\mcqgen\env\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [12]:
quiz_chain

LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. \nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make {number} MCQs\n### RESPONSE_JSON\n{response_json}\n\n'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000021CCE760AA0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000021CCE761460>, model_name='gemma-7b-it', temperature=0.3, groq_api_key=SecretStr('**********')), output_key='quiz')

In [13]:
TEMPLATE="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [14]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [15]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [16]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
   output_variables=["quiz", "review"], verbose=True,)

In [17]:
# """pdf_reader=PyPDF2.PdfReader(r"C:\Complete_Content\All_Project\TEST_FOR_EVERYTHING\langchain\experiments\Big Data Assessment.pdf")
# text = ""
# for page in pdf_reader.pages:
#     text += page.extract_text()"

In [18]:
from pathlib import Path
file_path=r"D:\mcqgen\data.txt"

In [19]:
file_path

'D:\\mcqgen\\data.txt'

In [20]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [21]:
TEXT

'Data science is an interdisciplinary academic field[1] that uses statistics, scientific computing, scientific methods, processes, algorithms and systems to extract or extrapolate knowledge and insights from noisy, structured, and unstructured data.[2]\n\nData science also integrates domain knowledge from the underlying application domain (e.g., natural sciences, information technology, and medicine).[3] Data science is multifaceted and can be described as a science, a research paradigm, a research method, a discipline, a workflow, and a profession.[4]\n\nData science is a "concept to unify statistics, data analysis, informatics, and their related methods" to "understand and analyze actual phenomena" with data.[5] It uses techniques and theories drawn from many fields within the context of mathematics, statistics, computer science, information science, and domain knowledge.[6] However, data science is different from computer science and information science. Turing Award winner Jim Gray

In [22]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [23]:
"""mcq_count=5
grade=1
tone="simple"""

'mcq_count=5\ngrade=1\ntone="simple'

In [24]:
NUMBER=4
SUBJECT="machine learning"
TONE="simple"

In [25]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

d:\mcqgen\env\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Data science is an interdisciplinary academic field[1] that uses statistics, scientific computing, scientific methods, processes, algorithms and systems to extract or extrapolate knowledge and insights from noisy, structured, and unstructured data.[2]

Data science also integrates domain knowledge from the underlying application domain (e.g., natural sciences, information technology, and medicine).[3] Data science is multifaceted and can be described as a science, a research paradigm, a research method, a discipline, a workflow, and a profession.[4]

Data science is a "concept to unify statistics, data analysis, informatics, and their related methods" to "understand and analyze actual phenomena" with data.[5] It uses techniques and theories drawn from many fields within the context of mathematics, statistics, computer science, information science, and domain knowledge.[6] However

In [26]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1636
Prompt Tokens:1145
Completion Tokens:491
Total Cost:0.0


In [27]:
response

{'text': 'Data science is an interdisciplinary academic field[1] that uses statistics, scientific computing, scientific methods, processes, algorithms and systems to extract or extrapolate knowledge and insights from noisy, structured, and unstructured data.[2]\n\nData science also integrates domain knowledge from the underlying application domain (e.g., natural sciences, information technology, and medicine).[3] Data science is multifaceted and can be described as a science, a research paradigm, a research method, a discipline, a workflow, and a profession.[4]\n\nData science is a "concept to unify statistics, data analysis, informatics, and their related methods" to "understand and analyze actual phenomena" with data.[5] It uses techniques and theories drawn from many fields within the context of mathematics, statistics, computer science, information science, and domain knowledge.[6] However, data science is different from computer science and information science. Turing Award winner

In [28]:
quiz_str=response.get("quiz")

In [35]:
quiz_str

'### RESPONSE_JSON\n{\n"1": {\n"mcq": "What is the primary focus of data science?",\n"options": {\n"a": "Developing computer programs",\n"b": "Extracting knowledge from data",\n"c": "Designing databases",\n"d": "Building complex algorithms",\n},\n"correct": "b"\n},\n"2": {\n"mcq": "Which of the following is not a component of data science?",\n"options": {\n"a": "Statistics",\n"b": "Philosophy",\n"c": "Algorithms",\n"d": "Data visualization",\n},\n"correct": "b"\n},\n"3": {\n"mcq": "Who imagined data science as a \'fourth paradigm\' of science?",\n"options": {\n"a": "Turing Award winner Jim Gray",\n"b": "Statistician Nathan Yau",\n"c": "Ben Fry",\n"d": "American Statistical Association",\n},\n"correct": "a"\n},\n"4": {\n"mcq": "What is the role of a data scientist?",\n"options": {\n"a": "Creating programming code",\n"b": "Combining programming code with statistical knowledge",\n"c": "Presenting findings to clients",\n"d": "Developing complex algorithms",\n},\n"correct": "b"\n}\n}'

In [29]:
print(quiz_str)

### RESPONSE_JSON
{
"1": {
"mcq": "What is the primary focus of data science?",
"options": {
"a": "Developing computer programs",
"b": "Extracting knowledge from data",
"c": "Designing databases",
"d": "Building complex algorithms",
},
"correct": "b"
},
"2": {
"mcq": "Which of the following is not a component of data science?",
"options": {
"a": "Statistics",
"b": "Philosophy",
"c": "Algorithms",
"d": "Data visualization",
},
"correct": "b"
},
"3": {
"mcq": "Who imagined data science as a 'fourth paradigm' of science?",
"options": {
"a": "Turing Award winner Jim Gray",
"b": "Statistician Nathan Yau",
"c": "Ben Fry",
"d": "American Statistical Association",
},
"correct": "a"
},
"4": {
"mcq": "What is the role of a data scientist?",
"options": {
"a": "Creating programming code",
"b": "Combining programming code with statistical knowledge",
"c": "Presenting findings to clients",
"d": "Developing complex algorithms",
},
"correct": "b"
}
}


In [31]:
# quiz_dict = json.loads(quiz_str)

In [36]:
# response_str = '''### RESPONSE_JSON
# {
# "1": {
# "mcq": "What is the primary focus of data science?",
# "options": {
# "a": "Developing computer programs",
# "b": "Extracting knowledge from data",
# "c": "Designing databases",
# "d": "Building complex algorithms"
# },
# "correct": "b"
# },
# "2": {
# "mcq": "Which of the following is not a component of data science?",
# "options": {
# "a": "Statistics",
# "b": "Philosophy",
# "c": "Algorithms",
# "d": "Data visualization"
# },
# "correct": "b"
# },
# "3": {
# "mcq": "Who imagined data science as a 'fourth paradigm' of science?",
# "options": {
# "a": "Turing Award winner Jim Gray",
# "b": "Statistician Nathan Yau",
# "c": "Ben Fry",
# "d": "American Statistical Association"
# },
# "correct": "a"
# },
# "4": {
# "mcq": "What is the role of a data scientist?",
# "options": {
# "a": "Creating programming code",
# "b": "Combining programming code with statistical knowledge",
# "c": "Presenting findings to clients",
# "d": "Developing complex algorithms"
# },
# "correct": "b"
# }
# }'''

# Step 1: Split the string into lines
lines = quiz_str.splitlines()

# Step 2: Initialize variables
mcq_dict = {}
current_mcq = {}
current_question = None

# Step 3: Loop through the lines and extract information
for line in lines:
    line = line.strip()
    
    if line.startswith('"mcq":'):
        current_question = line.split(':', 1)[1].strip().strip('",')
        current_mcq['mcq'] = current_question
        
    elif line.startswith('"options":'):
        current_mcq['options'] = {}
    
    elif '"a":' in line or '"b":' in line or '"c":' in line or '"d":' in line:
        option_key, option_value = line.split(':', 1)
        option_key = option_key.strip().strip('",')
        option_value = option_value.strip().strip('",')
        current_mcq['options'][option_key] = option_value
    
    elif line.startswith('"correct":'):
        correct_answer = line.split(':', 1)[1].strip().strip('",')
        current_mcq['correct'] = correct_answer
        mcq_dict[len(mcq_dict) + 1] = current_mcq
        current_mcq = {}

# Step 4: Output the structured data
import pprint
pprint.pprint(mcq_dict)


{1: {'correct': 'b',
     'mcq': 'What is the primary focus of data science?',
     'options': {'a': 'Developing computer programs',
                 'b': 'Extracting knowledge from data',
                 'c': 'Designing databases',
                 'd': 'Building complex algorithms'}},
 2: {'correct': 'b',
     'mcq': 'Which of the following is not a component of data science?',
     'options': {'a': 'Statistics',
                 'b': 'Philosophy',
                 'c': 'Algorithms',
                 'd': 'Data visualization'}},
 3: {'correct': 'a',
     'mcq': "Who imagined data science as a 'fourth paradigm' of science?",
     'options': {'a': 'Turing Award winner Jim Gray',
                 'b': 'Statistician Nathan Yau',
                 'c': 'Ben Fry',
                 'd': 'American Statistical Association'}},
 4: {'correct': 'b',
     'mcq': 'What is the role of a data scientist?',
     'options': {'a': 'Creating programming code',
                 'b': 'Combining programming

In [39]:
mcq_dict

{1: {'mcq': 'What is the primary focus of data science?',
  'options': {'a': 'Developing computer programs',
   'b': 'Extracting knowledge from data',
   'c': 'Designing databases',
   'd': 'Building complex algorithms'},
  'correct': 'b'},
 2: {'mcq': 'Which of the following is not a component of data science?',
  'options': {'a': 'Statistics',
   'b': 'Philosophy',
   'c': 'Algorithms',
   'd': 'Data visualization'},
  'correct': 'b'},
 3: {'mcq': "Who imagined data science as a 'fourth paradigm' of science?",
  'options': {'a': 'Turing Award winner Jim Gray',
   'b': 'Statistician Nathan Yau',
   'c': 'Ben Fry',
   'd': 'American Statistical Association'},
  'correct': 'a'},
 4: {'mcq': 'What is the role of a data scientist?',
  'options': {'a': 'Creating programming code',
   'b': 'Combining programming code with statistical knowledge',
   'c': 'Presenting findings to clients',
   'd': 'Developing complex algorithms'},
  'correct': 'b'}}

In [42]:
type(mcq_dict)

dict

In [43]:
mcq_dict.keys()

dict_keys([1, 2, 3, 4])

In [44]:
def create_mcq_dataframe(mcqs):
    flattened_data = []
    for num, mcq in mcqs.items():
        row = {
            'question_number': num,
            'question': mcq['mcq'],  # Update to access 'mcq' instead of 'question'
            'correct_answer': mcq['correct']  # Update to access 'correct' instead of 'correct_answer'
        }
        for option, text in mcq['options'].items():
            row[f'option_{option}'] = text
        flattened_data.append(row)
    
    return pd.DataFrame(flattened_data)

In [46]:
df = create_mcq_dataframe(mcq_dict)


In [47]:
df

,question_number,question,correct_answer,option_a,option_b,option_c,option_d
0,1,What is the primary focus of data science?,b,Developing computer programs,Extracting knowledge from data,Designing databases,Building complex algorithms
1,2,Which of the following is not a component of d...,b,Statistics,Philosophy,Algorithms,Data visualization
2,3,Who imagined data science as a 'fourth paradig...,a,Turing Award winner Jim Gray,Statistician Nathan Yau,Ben Fry,American Statistical Association
3,4,What is the role of a data scientist?,b,Creating programming code,Combining programming code with statistical kn...,Presenting findings to clients,Developing complex algorithms


In [48]:
df.to_csv("machine learning_Quiz.csv",index=False)